<a href="https://colab.research.google.com/github/Hedrios/analise-covid-brasil/blob/main/ANALISE_DE_CASOS_COVID_BRASIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANÁLISE DE CASOS COVID BRASIL**

# **1. Contexto**

> Este projeto visa a análise exploratória dos dados de casos de COVID-19 no Brasil, provenientes da universidade Johns Hopkins, bem como dos dados de vacinação, fornecidos pela universidade de Oxford. A análise inclui a extração, transformação e carregamento desses dados para construir um painel interativo que permita visualizar e compreender a evolução da pandemia no Brasil ao longo de 2021.

# **2. Pacotes e Bibliotecas**

> Utilizamos os seguintes pacotes e bibliotecas Python para processar e manipular os dados:

In [ ]:
# Funções matemáticas
import math

# Tipo para iteradores
from typing import Iterator

# Manipulação de datas
from datetime import datetime, timedelta

# Operações numéricas
import numpy as np

# Análise de dados
import pandas as pd


# **3. Extração de Dados**

## **3.1 Extração de Dados Diários**

>Os dados de casos de COVID-19 estão disponíveis em arquivos diários no repositório da Universidade Johns Hopkins. Cada arquivo representa os dados de um único dia, contendo informações de vários países, incluindo o Brasil.

> Por exemplo, para carregar os dados do dia 1º de dezembro de 2021, podemos utilizar o seguinte código:

In [ ]:
# Exemplo de como carregar os dados de 1º de dezembro de 2021
cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/12-01-2021.csv', sep=',')

# Visualizando as primeiras linhas do arquivo de 1º de dezembro de 2021
cases.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-12-02 04:22:37,33.93911,67.709953,157359,7309,NaN,NaN,Afghanistan,404.227347,4.644793
1,NaN,NaN,NaN,Albania,2021-12-02 04:22:37,41.15330,20.168300,200173,3101,NaN,NaN,Albania,6955.764820,1.549160
2,NaN,NaN,NaN,Algeria,2021-12-02 04:22:37,28.03390,1.659600,210723,6076,NaN,NaN,Algeria,480.542732,2.883406
3,NaN,NaN,NaN,Andorra,2021-12-02 04:22:37,42.50630,1.521800,17426,131,NaN,NaN,Andorra,22553.549473,0.751750
4,NaN,NaN,NaN,Angola,2021-12-02 04:22:37,-11.20270,17.873900,65183,1735,NaN,NaN,Angola,198.327963,2.661737


> Esse exemplo ilustra como acessar os dados de um único dia. No entanto, para coletar todos os dados de 2021, precisamos iterar sobre todas as datas do ano e consolidar as informações em um único DataFrame.

## **3.2. Extração de Dados de Casos**

> Os dados de casos estão compilados em arquivos diários no repositório da Johns Hopkins. Para capturá-los, utilizamos uma função que itera sobre um intervalo de datas de 1 de janeiro de 2021 até 31 de dezembro de 2021.

### **3.2.1. Definição de Intervalo de Tempo**

> Criando uma função **date_range** para gerar as datas que cobrem o período de análise

In [ ]:
# Função para gerar um intervalo de datas entre start_date e end_date
def date_range(start_date: datetime, end_date: datetime) -> Iterator[datetime]:
    # Calcula o número de dias entre as datas e retorna uma data a cada iteração
    date_range_days: int = (end_date - start_date).days
    for lag in range(date_range_days):
        yield start_date + timedelta(lag)


> Definir o intervalo de tempo de interesse:

In [ ]:
# Definindo o intervalo de datas de 1 de janeiro de 2021 até 31 de dezembro de 2021
start_date = datetime(2021, 1, 1)
end_date   = datetime(2021, 12, 31)


### **3.2.2. Iteração sobre as Datas**

> Em seguida, iteramos pelas datas e carregamos os dados de casos diários, aplicando as transformações iniciais para selecionar as colunas relevantes e filtrar os dados do Brasil.

In [ ]:
# Inicializando variáveis para armazenar os dados de casos
cases = None
cases_is_empty = True

# Itera sobre cada data no intervalo e coleta os dados de COVID-19
for date in date_range(start_date=start_date, end_date=end_date):
    # Formata a data no padrão MM-DD-YYYY para acessar o arquivo correto
    date_str = date.strftime('%m-%d-%Y')

    # Monta a URL para acessar o arquivo CSV com os dados diários de COVID-19
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

    # Lê os dados do CSV da URL
    case = pd.read_csv(data_source_url, sep=',')

    # REMOVE COLUNAS QUE NÃO SÃO NECESSÁRIAS PARA A ANÁLISE
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)

    # Filtra os dados para o Brasil
    case = case.query('Country_Region == "Brazil"').reset_index(drop=True)

    # Adiciona uma coluna com a data correspondente
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

    # Se a variável cases estiver vazia, inicializa com o primeiro DataFrame, caso contrário concatena os dados
    if cases_is_empty:
        cases = case
        cases_is_empty = False
    else:
        cases = pd.concat([cases, case], axis=0, ignore_index=True)

# Visualizando as primeiras linhas dos dados de casos
cases.head()


,Province_State,Country_Region,Confirmed,Deaths,Incident_Rate,Date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


## **3.3. Extração de Dados de Vacinação**

> Os dados de vacinação estão compilados em um único arquivo, disponível no repositório da universidade de Oxford. Selecionamos as colunas de interesse para o Brasil.

In [ ]:
# Carregando os dados de vacinação
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3])

# Filtrando os dados de vacinação para o Brasil
vaccines = vaccines.query('location == "Brazil"').reset_index(drop=True)

# Selecionando apenas as colunas de interesse
vaccines = vaccines[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

# Visualizando as primeiras linhas dos dados de vacinação
vaccines.head()


,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


# **4. Transformação de Dados**

> Nesta etapa, transfo os dados extraídos para garantir que estejam em um formato adequado para análise e visualização.

## **4.1. Transformação dos Dados de Casos**

### **4.1.1. Renomeação de Colunas**

In [ ]:
# Renomeia as colunas "Province_State" e "Country_Region" para "state" e "country" para melhor leitura
cases = cases.rename(
    columns={
        'Province_State': 'state',
        'Country_Region': 'country'
    }
)

# Renomeia todas as colunas para letras minúsculas
for col in cases.columns:
    cases = cases.rename(columns={col: col.lower()})

# Visualizando as primeiras linhas após a renomeação das colunas
cases.head()


,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapa,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


### **4.1.2. Ajustes de Nomes de Estados**

In [ ]:
# Dicionário de mapeamento para corrigir nomes de estados com ortografia incorreta
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

# Corrige os nomes dos estados com base no dicionário de mapeamento
cases['state'] = cases['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

# Visualizando as primeiras linhas após o ajuste de nomes de estados
cases.head()


,state,country,confirmed,deaths,incident_rate,date
0,Acre,Brazil,41689,796,4726.992352,2021-01-01
1,Alagoas,Brazil,105091,2496,3148.928928,2021-01-01
2,Amapá,Brazil,68361,926,8083.066602,2021-01-01
3,Amazonas,Brazil,201574,5295,4863.536793,2021-01-01
4,Bahia,Brazil,494684,9159,3326.039611,2021-01-01


### **4.1.3. Criação de Novas Colunas**

> Criando colunas temporais e calculamos estimativas de população e outros indicadores.

In [ ]:
# Cria colunas com chaves temporais: mês e ano
cases['month'] = cases['date'].apply(lambda date: date.strftime('%Y-%m'))
cases['year']  = cases['date'].apply(lambda date: date.strftime('%Y'))

# Calcula a estimativa de população com base nos casos confirmados e taxa de incidentes
cases['population'] = round(100000 * (cases['confirmed'] / cases['incident_rate']))

# Remove a coluna incident_rate, pois não é mais necessária
cases = cases.drop('incident_rate', axis=1)

# Visualizando as primeiras linhas após a criação de novas colunas
cases.head()


,state,country,confirmed,deaths,date,month,year,population
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0
1,Alagoas,Brazil,105091,2496,2021-01-01,2021-01,2021,3337357.0
2,Amapá,Brazil,68361,926,2021-01-01,2021-01,2021,845731.0
3,Amazonas,Brazil,201574,5295,2021-01-01,2021-01,2021,4144597.0
4,Bahia,Brazil,494684,9159,2021-01-01,2021-01,2021,14873064.0


### **4.1.4. Cálculos de Médias Móveis e Estabilidade**

>Para analisar as tendências de casos e mortes por estado, aplicamos os seguintes cálculos:



*   **Média móvel de 7 dias:** Calculamos a média móvel para suavizar as flutuações diárias nos números de casos e mortes.

*   **Estabilidade (14 dias):** Comparando a média móvel atual com a média de 14 dias atrás, podemos determinar a tendência: se está subindo, descendo ou estável.

>A tabela abaixo ilustra a lógica de como esses cálculos são realizados ao longo do tempo:



| 1 | 2 | 3 | 4 | 5 | 6 | <font color='red'>7</font> | <font color='green'>8</font> | 9 | 10 | 11 | 12 | 13 | <font color='blue'>14<font color='red'> | 15 | 16 | 17 | 18 | 19 | 20 | 21 |
| - | - | - | - | - | - | - | - | - | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| <font color='red'>D-6</font> | <font color='red'>D-5</font> | <font color='red'>D-4</font> | <font color='red'>D-3</font> | <font color='red'>D-2</font> | <font color='red'>D-1</font> | <font color='red'>D0</font> | | | | | | | | | | | | | | |
| D-7 | <font color='green'>D-6</font> | <font color='green'>D-5</font> | <font color='green'>D-4</font> | <font color='green'>D-3</font> | <font color='green'>D-2</font> | <font color='green'>D-1</font> | <font color='green'>D0</font> | | | | | | | | | | | | | |
| D-13 | D-12 | D-11 | D-10 | D-9 | D-8 | D-7 | <font color='blue'>D-6</font> | <font color='blue'>D-5</font> | <font color='blue'>D-4</font> | <font color='blue'>D-3</font> | <font color='blue'>D-2</font> | <font color='blue'>D-1</font> | <font color='blue'>D0</font> | | | | | | | |

A lógica programática para esse cálculo já foi implementada no código abaixo:

In [ ]:
# Inicializando a variável cases_ como um DataFrame vazio
cases_ = pd.DataFrame()

# Função para determinar a tendência dos casos e óbitos (decrescente, crescente ou estável)
def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN
    if rate < 0.75:  # Se a taxa for menor que 0.75, a tendência é decrescente
        return 'downward'
    elif rate > 1.15:  # Se a taxa for maior que 1.15, a tendência é crescente
        return 'upward'
    else:
        return 'stable'  # Caso contrário, a tendência é estável

# Itera sobre cada estado e calcula médias móveis e tendências para os casos e óbitos
for state in cases['state'].drop_duplicates():
    # Filtra os dados por estado e ordena pela data
    cases_per_state = cases.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    # Calcula o número de casos confirmados por dia
    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)

    # Calcula a média móvel de 7 dias para casos confirmados
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())

    # Calcula a taxa de variação de 14 dias da média móvel de casos confirmados
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d'] / cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)

    # Aplica a função para calcular a tendência dos casos confirmados
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    # Calcula o número de óbitos por dia
    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)

    # Calcula a média móvel de 7 dias para óbitos
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())

    # Calcula a taxa de variação de 14 dias da média móvel de óbitos
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d'] / cases_per_state['deaths_moving_avg_7d'].shift(periods=14)

    # Aplica a função para calcular a tendência dos óbitos
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    # Concatenando os dados processados no DataFrame cases_
    cases_ = pd.concat([cases_, cases_per_state], axis=0, ignore_index=True)

# Substitui a variável original "cases" pelos dados processados
cases = cases_

# Visualizando as primeiras linhas após o cálculo de médias móveis e tendências
cases.head()


,state,country,confirmed,deaths,date,month,year,population,confirmed_1d,confirmed_moving_avg_7d,confirmed_moving_avg_7d_rate_14d,confirmed_trend,deaths_1d,deaths_moving_avg_7d,deaths_moving_avg_7d_rate_14d,deaths_trend
0,Acre,Brazil,41689,796,2021-01-01,2021-01,2021,881935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Acre,Brazil,41941,798,2021-01-02,2021-01,2021,881935.0,252.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2,Acre,Brazil,42046,802,2021-01-03,2021-01,2021,881935.0,105.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,Acre,Brazil,42117,806,2021-01-04,2021-01,2021,881935.0,71.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN
4,Acre,Brazil,42170,808,2021-01-05,2021-01,2021,881935.0,53.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN


## **4.2. Transformação dos Dados de Vacinação**

### **4.2.1. Tratamento de Dados Faltantes**

> Prenchendo os dados faltantes com o valor mais próximo anterior.

In [ ]:
# Aplicando o preenchimento de valores ausentes apenas nas colunas que podem ter NaN
columns_to_fill = ['total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters']
vaccines[columns_to_fill] = vaccines[columns_to_fill].ffill()

# Visualizando as primeiras linhas após o preenchimento de valores faltantes
vaccines.head()



,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-05
1,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-06
2,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-07
3,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-08
4,Brazil,215313504,NaN,NaN,NaN,NaN,2020-01-09


### **4.2.2. Filtragem por Período de Tempo**

> Filtrando os dados para garantir que tratam do mesmo período dos dados de casos.

In [ ]:
# Filtrando os dados de vacinação para o mesmo período dos casos (2021)
vaccines = vaccines[(vaccines['date'] >= '2021-01-01') & (vaccines['date'] <= '2021-12-31')].reset_index(drop=True)

# Visualizando as primeiras linhas após a filtragem por período
vaccines.head()


,location,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,date
0,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-01
1,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-02
2,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-03
3,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-04
4,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-05


### **4.2.3. Criação de Novas Colunas**

> Criando as colunas para indicar as porcentagens da população vacinada.

In [ ]:
# Renomeando colunas de vacinação para facilitar a leitura
vaccines = vaccines.rename(
    columns={
        'location': 'country',
        'total_vaccinations': 'total',
        'people_vaccinated': 'one_shot',
        'people_fully_vaccinated': 'two_shots',
        'total_boosters': 'three_shots',
    }
)

# Criando colunas para a chave temporal (mês e ano)
vaccines['month'] = vaccines['date'].apply(lambda date: date.strftime('%Y-%m'))
vaccines['year']  = vaccines['date'].apply(lambda date: date.strftime('%Y'))

# Calculando as porcentagens de pessoas vacinadas (uma dose, duas doses, e reforço)
vaccines['one_shot_perc'] = round(vaccines['one_shot'] / vaccines['population'], 4)
vaccines['two_shots_perc'] = round(vaccines['two_shots'] / vaccines['population'], 4)
vaccines['three_shots_perc'] = round(vaccines['three_shots'] / vaccines['population'], 4)

# Visualizando as primeiras linhas após a criação de novas colunas
vaccines.head()


,country,population,total,one_shot,two_shots,three_shots,date,month,year,one_shot_perc,two_shots_perc,three_shots_perc
0,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-01,2021-01,2021,NaN,NaN,NaN
1,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-02,2021-01,2021,NaN,NaN,NaN
2,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-03,2021-01,2021,NaN,NaN,NaN
3,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-04,2021-01,2021,NaN,NaN,NaN
4,Brazil,215313504,NaN,NaN,NaN,NaN,2021-01-05,2021-01,2021,NaN,NaN,NaN


# **5. Carregamento**

> Após manipular os dados de casos e vacinação, salva-se os arquivos para posterior carregamento no Google Data Studio

## **5.1. Salvamento dos Dados Processados**

### **5.1.1. Casos de COVID-19**

In [ ]:
# Salvando os dados de casos da COVID-19 em um arquivo CSV
cases.to_csv('./covid-cases.csv', sep=',', index=False)

### **5.1.2. Dados de Vacinação**

In [ ]:
# Salvando os dados de vacinação da COVID-19 em um arquivo CSV
vaccines.to_csv('./covid-vaccines.csv', sep=',', index=False)


# **6. Dashboard Interativo**


Foi criado um **dashboard interativo** no **Google Data** Studio para facilitar a visualização dos dados de COVID-19 no Brasil. O painel apresenta as principais métricas, como:



*   **Casos confirmados e óbitos:** Exibidos tanto para as últimas 24 horas quanto como médias móveis de 7 dias.

*   **Indicadores de tendências:** Mostra se os casos e mortes estão estáveis, subindo ou caindo, com base em análises de variação de 14 dias.

*   **Vacinação:** Percentuais de pessoas vacinadas com a primeira, segunda e terceira doses.


**O mapa interativo** permite que os usuários cliquem em diferentes estados para ver informações detalhadas sobre casos, mortes e cobertura vacinal. Além disso, gráficos de linha e barras mostram a evolução dos casos e mortes ao longo do tempo.


Com a interatividade oferecida pelo Google Data Studio, é possível filtrar os dados por estado e período, proporcionando uma análise rápida e eficiente da pandemia.



Você pode acessar o **LINK:**
https://lookerstudio.google.com/u/2/reporting/94f4db85-55cd-486e-89ca-ba417eaff0c0/page/wYXFE






# **7. Conclusão**

Este projeto organizou e analisou dados de casos de COVID-19 e vacinação no Brasil durante o ano de 2021. Utilizando ferramentas como Python e Pandas, foi possível realizar uma série de transformações nos dados, que permitiram calcular tendências de casos, médias móveis e a taxa de variação ao longo de um período de 14 dias. Esses insights proporcionam uma visão detalhada da evolução da pandemia no Brasil, tanto em nível nacional quanto por estado.

Os resultados desta análise foram exportados e visualizados em um dashboard interativo criado no Google Data Studio (Item 6), facilitando a exploração dos dados em tempo real. O painel oferece uma interface clara e acessível, onde é possível clicar em estados e acessar informações sobre vacinação, óbitos, casos confirmados e recuperações de forma visual e intuitiva.

Com isso, o projeto não só fornece uma análise profunda dos dados, como também oferece uma plataforma visual poderosa, ajudando a transformar dados brutos em insights acionáveis para tomada de decisão.